In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

# Get (Simple) VGG model

https://medium.com/tensorflow/neural-style-transfer-creating-art-with-deep-learning-using-tf-keras-and-eager-execution-7d541ac31398

https://keras.io/examples/generative/neural_style_transfer/

In [2]:
def get_model():
    vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    # config for NST laters
    return vgg

In [3]:
model = get_model()

80142336/80134624 [==============================] - 10s 0us/step


In [4]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [5]:
model.get_layer("block5_conv1")

In [6]:
del model

# Get VGG for Neural Style Transfer

In [7]:
# neural style transfer output

content_layers = ['block5_conv2']
style_layers = [
    'block1_conv1',
    'block2_conv1',
    'block3_conv1',
    'block4_conv1',
    'block5_conv1',
]

In [8]:
def get_nst_feature_extractor(content_layers, style_layers):
    vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False

    # extracted feature
    layers = content_layers + style_layers
    outputs_dict = dict([(layer, vgg.get_layer(layer).output) for layer in layers])
    
    # model for nst
    feature_extractor = tf.keras.Model(inputs=vgg.input, outputs=outputs_dict)
    
    return feature_extractor

In [9]:
nst_model = get_nst_feature_extractor(content_layers, style_layers)

In [10]:
nst_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

# Example

In [11]:
im = Image.open("./img/2valk.jpg")
arr_im = np.array(im)

In [12]:
arr_im.shape

(1818, 2048, 3)

In [13]:
def model_output_to_numpy(model_output):
    for key in model_output.keys():
        model_output[key] = np.array(model_output[key])[0]
    return model_output

In [14]:
model_output = nst_model(np.expand_dims(arr_im, axis=0))

ResourceExhaustedError: OOM when allocating tensor with shape[1,64,1818,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

In [ ]:
model_output = model_output_to_numpy(model_output)

In [ ]:
model_output.keys()

In [ ]:
model_output['block1_conv1']

## Visualize Model output activation/feature map

In [ ]:
layer_outputs = model_output['block1_conv1']
layer_outputs.shape

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.imshow(arr_im)
plt.show()

In [ ]:
for i in range(layer_outputs.shape[2]):
    if i % 5 == 0:
        fig = plt.figure(figsize=(5, 5))
        plt.imshow(layer_outputs[:, :, i], cmap=plt.get_cmap('gray'))
        plt.show()